Daniel Rocha Ruiz, MSc in Data Science and Business Analytics

Sources:
- https://www.datacamp.com/community/tutorials/geospatial-data-python

Data:
- Geospatial shapes:
    - https://eric.clst.org/tech/usgeojson/
- Hurricane Florence:
    - http://flhurricane.com/cyclone/stormhistory.php?storm=6&year=2018

# Loading packages
The package geopandas can be a bit tricky to install and require Microsoft C++ Build tools to be installed in your machine.
- https://visualstudio.microsoft.com/visual-cpp-build-tools/

In [1]:
# geospatial
import geopandas
from shapely.geometry import Point

# maths
import numpy as np
import pandas as pd

# viz
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'geopandas'

In [6]:
!pip install geopandas

  Using cached geopandas-0.10.2-py2.py3-none-any.whl (1.0 MB)
  Using cached Fiona-1.8.21.tar.gz (1.0 MB)
  Using cached Fiona-1.8.20.tar.gz (1.3 MB)
  Using cached Fiona-1.8.19.tar.gz (1.3 MB)
  Using cached Fiona-1.8.18.tar.gz (1.3 MB)
  Using cached Fiona-1.8.17.tar.gz (1.3 MB)
  Using cached Fiona-1.8.16.tar.gz (1.3 MB)
  Using cached Fiona-1.8.15.tar.gz (1.3 MB)
  Using cached Fiona-1.8.14.tar.gz (1.3 MB)
  Using cached Fiona-1.8.13.post1.tar.gz (1.2 MB)
  Using cached Fiona-1.8.13.tar.gz (1.2 MB)
  Using cached Fiona-1.8.12.tar.gz (1.2 MB)
  Using cached Fiona-1.8.11.tar.gz (1.2 MB)
  Using cached Fiona-1.8.10.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.post2.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.post1.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.tar.gz (1.2 MB)
  Using cached Fiona-1.8.8.tar.gz (1.7 MB)
  Using cached Fiona-1.8.7.tar.gz (1.7 MB)
  Using cached Fiona-1.8.6.tar.gz (1.7 MB)
  Using cached Fiona-1.8.5.tar.gz (1.7 MB)
  Using cached Fiona-1.8.4.tar.gz (1.1 MB)
  Us

    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Miniconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\drr19\\AppData\\Local\\Temp\\pip-install-ixuvii86\\fiona_6b555d1afb904e5e841c5d9f08efd3ca\\setup.py'"'"'; __file__='"'"'C:\\Users\\drr19\\AppData\\Local\\Temp\\pip-install-ixuvii86\\fiona_6b555d1afb904e5e841c5d9f08efd3ca\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\drr19\AppData\Local\Temp\pip-pip-egg-info-odvs80bh'
         cwd: C:\Users\drr19\AppData\Local\Temp\pip-install-ixuvii86\fiona_6b555d1afb904e5e841c5d9f08efd3ca\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment

# 2. Loading map canvas

In [ ]:
country = geopandas.read_file("gz_2010_us_040_00_5m.json")
country.head()

In [ ]:
# new objects
# shapely is geometrical!
print(type(country))
print(type(country.geometry))
print(type(country.geometry[0]))

In [ ]:
# built in plot
country.plot(figsize=(20,20),color='#3B3C6E')
plt.show()

In [ ]:
# exclude Alaska and Hawaii (for now)
country[country['NAME'].isin(['Alaska','Hawaii']) == False].plot(figsize=(30,20), color='#3B3C6E')
plt.show()

# 3. Loading hurricane data

In [ ]:
# load hurricane data
florence = pd.read_csv('florence.txt')

# basic data info -> one missing observation
print(florence.info())
print(florence.describe())

# basic data
print('Mean wind speed of Hurricane Florence (kpm): {:.2f}'.format(round(1.61*florence.Wind.mean(),4)))
print('Max wind speed of Hurricane Florence (kpm): {:.2f}'.format(1.61*florence.Wind.max()))

# beware, the hurricane was given 4 different names
print(florence.groupby('Name').Type.count())

In [ ]:
# drop variables that will not be used
florence = florence.drop(['AdvisoryNumber', 'Forecaster', 'Received'], axis=1)

# add a '-' before longitude (so you can see lat-long data as 00-00)
florence['Long'] = 0 - florence['Long']

# combine lattitude and longitude into coordinates
florence['coordinates'] = florence[['Long', 'Lat']].values.tolist()
florence['coordinates'] = florence['coordinates'].apply(Point)

# view
print(type(florence))
print(type(florence['coordinates']))

# convert to geodataframe
florence = geopandas.GeoDataFrame(florence, geometry='coordinates')
print(type(florence))
print(type(florence['coordinates']))
florence.head()

# 4. Ploting in map

In [ ]:
# plot the hurricane's trajectory
florence.plot(figsize=(20,10))
plt.plot()

In [ ]:
# plot the hurricane's trajectory on the US map
fig, ax = plt.subplots(1, figsize=(30,20))
base = country[country['NAME'].isin(['Alaska','Hawaii']) == False].plot(ax=ax, color='#3B3C6E')

# edit marker
florence.plot(ax=base, color='red', marker="*", markersize=20)

In [ ]:
# create figure
fig, ax = plt.subplots(1, figsize=(20,20))
base = country[country['NAME'].isin(['Alaska','Hawaii']) == False].plot(ax=ax, color='#3B3C6E')

# trajectory color based on windspeed, remove axis
florence.plot(ax=base, column='Wind', marker="<", markersize=10, cmap='cool', label="Wind speed(mph)")
_ = ax.axis('off')

# add title, legend
ax.set_title("Hurricane Florence in US Map", fontsize=25)
plt.legend()

# save
#plt.savefig('Hurricane_footage.png',bbox_inches='tight')
plt.show()

# 5. Wrap-up

In [ ]:
# getting data
country = geopandas.read_file("gz_2010_us_040_00_5m.json")
florence = pd.read_csv('florence.txt')

# editing dataframe
florence = florence.drop(['AdvisoryNumber', 'Forecaster', 'Received'], axis=1)
florence['Long'] = 0 - florence['Long']
florence['coordinates'] = florence[['Long', 'Lat']].values.tolist()
florence['coordinates'] = florence['coordinates'].apply(Point)
florence = geopandas.GeoDataFrame(florence, geometry='coordinates')

# ploting
fig, ax = plt.subplots(1, figsize=(20,20))
base = country[country['NAME'].isin(['Alaska','Hawaii']) == False].plot(ax=ax, color='#3B3C6E')
florence.plot(ax=base, column='Wind', marker="<", markersize=10, cmap='cool', label="Wind speed(mph)")
_ = ax.axis('off')
ax.set_title("Hurricane Florence in US Map", fontsize=25)
plt.legend()
plt.show()